In [18]:
# 夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をMeCabを使って形態素解析し，
# その結果をneko.txt.mecabというファイルに保存せよ．
# このファイルを用いて，以下の問に対応するプログラムを実装せよ．

import requests

r = requests.get("https://nlp100.github.io/data/neko.txt", auth=('user','pass'))
r.status_code
r.headers['content-type']
r.encoding
with open("../data/neko.txt",mode='w') as f:
    f.write(r.text)

In [18]:
import MeCab
chasen = MeCab.Tagger("")
chasen.parse("")
with open("../data/neko.txt") as f:
    with open("../data/neko.txt.mecab",mode='w') as w_f:
        read_f = f.read()
        w_f.write(chasen.parse(read_f))

## 30. 形態素解析結果の読み込み

In [2]:
# 形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．
# ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，
# 1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

## 新しい   形容詞,自立,*,*,形容詞・イ段,基本形,新しい,アタラシイ,アタラシイ
## surface:新しい   pos:形容詞   pos1:自立 pos2:   pos3:   cform:形容詞・イ段    ctype:基本形 base:新しい  read:アタラシイ    pron:アタラシイ

with open("../data/neko.txt.mecab",mode='r') as r_f:
    text = r_f.read().split("\n")
    
result = []
for t_line in text:
    if t_line == 'EOS':
        continue
    t_array = t_line.split('\t')
    surface = t_array[0]
    key = t_array[1].split(",")
    dic = {'surface':surface, 'base':key[6], 'pos':key[0], 'pos1':key[1]}
    result.append(dic)
#result

## 31. 動詞

In [7]:
# 動詞の表層形をすべて抽出せよ．
for text_dic in result:
    if text_dic['pos'] == "動詞":
        #print(text_dic['surface'],text_dic['pos'])

SyntaxError: unexpected EOF while parsing (<ipython-input-7-4c0f622d5754>, line 4)

## 32. 動詞の基本形

In [12]:
# 動詞の基本形をすべて抽出せよ．
for text_dic in result:
    if text_dic['pos'] == "動詞":
        #print(text_dic['base'],text_dic['surface'])

IndentationError: expected an indented block (<ipython-input-12-daa017a60f88>, line 5)

## 33. 「AのB」

In [12]:
for index in range(len(result)):
    if (result[index]['pos1'] == "連体化") and (result[index]['surface'] == "の"):
        if (result[index-1]['pos'] == "名詞") and (result[index+1]['pos'] == "名詞"):
            #print(result[index-1]["surface"]+result[index]["surface"]+result[index+1]["surface"])

IndentationError: expected an indented block (<ipython-input-12-42a638813833>, line 5)

## 34. 名詞の連接

In [9]:
ans_1 = []
ans = []
for d in result:
    if d['pos'] == '名詞':
        ans_1.append(d['surface'])
    else:
        if len(ans_1) >= 2:
            ans_1_str = "".join(ans_1)
            ans.append(ans_1_str)
            ans_1 = []
        else:
            ans_1 = []

## 35. 単語の出現頻度